<a href="https://colab.research.google.com/github/as9786/ComputerVision/blob/main/model/code/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residual Network 구현 및 학습

In [1]:
import tensorflow as tf 
import numpy as np

# 초매개변수

In [2]:
EPOCHS = 10

# Residual Unit 구현

In [10]:
class ResidualUnit(tf.keras.Model):
  def __init__(self,filter_in,filter_out,kernel_size):
    super(ResidualUnit,self).__init__()
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

    self.bn2 = tf.keras.layers.BatchNormalization()
    self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

    if filter_in == filter_out: 
      self.identity = lambda x: x
    else:
      self.identity = tf.keras.layers.Conv2D(filter_out,(1,1),padding='same')

  def call(self, x, training=False, mask=None):
    h = self.bn1(x,training=training) 
    h = tf.nn.relu(h)
    h = self.conv1(h)

    h = self.bn2(h,training=training)
    h = tf.nn.relu(h)
    h = self.conv2(h)
    return self.identity(x) + h

# Residual Layer 구현

In [25]:
class ResnetLayer(tf.keras.Model):
  def __init__(self,filter_in,filters,kernel_size):
    super(ResnetLayer,self).__init__()
    self.sequence = list()
    for f_in, f_out in zip([filter_in] + list(filters),filters):
      # [16] + [32,32,32]
      # zip([16,32,32,32],[32,32,32]) 마지막 남은 것은 무시(작은것을 기준으로 zip)
      self.sequence.append(ResidualUnit(f_in,f_out,kernel_size))

  def call(self,x,training=False,mask=None):
      for unit in self.sequence:
        x = unit(x,training=training)

      return x

# 모형 정의

In [26]:
class ResNet(tf.keras.Model):
  def __init__(self):
    super(ResNet,self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(8,(3,3),padding='same',activation='relu')

    self.res1 = ResnetLayer(8,(16,16),(3,3)) 
    self.pool1 = tf.keras.layers.MaxPool2D((2,2))

    self.res2 = ResnetLayer(16,(32,32),(3,3)) 
    self.pool2 = tf.keras.layers.MaxPool2D((2,2))

    self.res3 = ResnetLayer(32,(64,64),(3,3))
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(128,activation='relu')
    self.dense2 = tf.keras.layers.Dense(10,activation='softmax')

  def call(self,x,training=False,mask=None):
    x = self.conv1(x)

    x = self.res1(x,training=training)
    x = self.pool1(x)
    x = self.res2(x,training=training)
    x = self.pool2(x)
    x = self.res3(x,training=training)  

    x = self.flatten(x)
    x = self.dense1(x)

    return self.dense2(x)


#Train and Test

## Dataset

In [27]:
mnist = tf.keras.datasets.mnist

(X_train,y_train),(X_test,y_test) = mnist.load_data()

X_train, X_test = X_train / 255., X_test / 255.

X_train = X_train[...,tf.newaxis].astype(np.float32)
X_test = X_test[...,tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(32)

## 학습 환경 정의

### 모형 생성, 손실함수, 최적화, 평가지표

In [28]:
model = ResNet() 

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [29]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images,training=True) 
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)

@tf.function
def test_step(model,images,labels,loss_object,test_loss,test_accuracy): 
  predictions = model(images,training=False)
  
  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)


### 학습

In [30]:
for epoch in range(EPOCHS): 
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds: 
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch {}, Loss : {}, Accuracy : {}, Test Loss : {}, Test Accuracy : {}'
  print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(),
                        test_accuracy.result() * 100))

  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


Epoch 1, Loss : 0.14186105132102966, Accuracy : 96.05166625976562, Test Loss : 0.05929366126656532, Test Accuracy : 98.25
Epoch 2, Loss : 0.06797344237565994, Accuracy : 98.163330078125, Test Loss : 0.09493330866098404, Test Accuracy : 96.91000366210938
Epoch 3, Loss : 0.04863991588354111, Accuracy : 98.63333129882812, Test Loss : 0.05782311409711838, Test Accuracy : 98.32999420166016
Epoch 4, Loss : 0.038828469812870026, Accuracy : 98.91999816894531, Test Loss : 0.04827779904007912, Test Accuracy : 98.88999938964844
Epoch 5, Loss : 0.03397541493177414, Accuracy : 99.00166320800781, Test Loss : 0.03204186260700226, Test Accuracy : 99.0199966430664
Epoch 6, Loss : 0.029959412291646004, Accuracy : 99.16333770751953, Test Loss : 0.052017614245414734, Test Accuracy : 98.63999938964844
Epoch 7, Loss : 0.026574017480015755, Accuracy : 99.25, Test Loss : 0.0369199700653553, Test Accuracy : 99.01000213623047
Epoch 8, Loss : 0.02378990687429905, Accuracy : 99.33666229248047, Test Loss : 0.05241